In [7]:
import re

import numpy as np
import pandas as pd


In [4]:
REPORTING_DATA_FOLDER = 'parsed_data/bank_reporting/'
CLOSED_BANKS_DATA_FOLDER = 'parsed_data/closed_banks/'
MANUALLY_PARSED_DATA_FOLDER = 'manually_collected_data/'

# Объединим все показатели в один датафрей

## Данные, которые нужны для модели

Сопоставление критериев и их ID:

+ Активы нетто: na     10
+ Чистая прибыль ni    30
(excel) + Капитал (по форме 123) c_123     25
(excel) + Капитал (по форме 134) c_134     20
+ Кредитный портфель credit_portf   40
+ Просроченная задолженность в кредитном портфеле sunk_credit_portf   50
+ Вклады физических лиц retail_deposit      60
+ Средства предприятий и организаций organization_deposit   500
+ Рентабельность активов-нетто ROA    1000
+ Рентабельность капитала ROE         1100 
     + Уровень просроченной задолженности по кредитному портфелю sunk_credit_perc   1200
     + Уровень резервирования по кредитному портфелю reserv_credit_perc    1300
     + Уровень обеспечения кредитного портфеля залогом имущества zalog_credit_perc      1400
     + Валютный оборот к активам-нетто foreign_na            1500
     + Н1 H1    1600
     + Н2 H2    1700
     + Н3 H3    1800
+ Кредиты физическим лицам retail_credit    200
+ Просроченная задолженность по кредитам физическим лицам sunk_retail_credit   260
+ Кредиты предприятиям и организациям organization_credit   300
+ Просроченная задолженность по кредитам предприятиям и организациям sunk_organization_credit_perc   360
+ Выданные МБК ibl   120

+ вложения в ценные бумаги:sec_tot 70 = подверженность рискам на фин рынках
+ высоколиквидные активы: liquid 110
+ вложения в акции sec 130
+ вложения в облигации bond 140
+ вложения в капиталы других организаций oth_cap 160
+ вклады физических лиц retail 400
     + вклады физических лиц оборот retail_deposit_fr 401

На Мар-шаге мы скачали все необходимые нам показатели с сайта в кучу маленьких csv-табличек, которые нам теперь нужно переработать в одну большую таблицу, которую мы будем впоследствии использовать при оценки моделей. Самого по себе Reduce-шага здесь не будет. Будет Panel - шаг!

Мы хотим, чтобы в итоговой таблице оформление было таким, чтобы было удобно в python работать с панельными данными. Итоговая таблица должна идти вот в таком формате:



| Банк   | Год   | Показатель 1 | Показатель 2 | Показатель 3 |
|--------|-------|--------------|--------------|--------------|
| Банк 1 | Год 1 |              |              |              |
| Банк 1 | Год 2 |              |              |              |
| Банк 2 | Год 1 |              |              |              |
| Банк 2 | Год 2 |              |              |              |

In [80]:
# Подгружаем списки из банков.
unique_banks = list(pd.read_csv(REPORTING_DATA_FOLDER + 'unique_banks.csv', sep='\t',
                              header = 0, encoding="cp1251").iloc[:,0])
unique_banks = [re.sub("\D",'',item) for item in unique_banks]

print(len(unique_banks))

# Подгружаем данные по всякой отозванной фигне. 
likvid_data = pd.read_csv(CLOSED_BANKS_DATA_FOLDER + 'likvid_data.csv',sep='\t',header=0, encoding="cp1251")
otozv_data = pd.read_csv(CLOSED_BANKS_DATA_FOLDER + 'otozv_data.csv',sep='\t',header=0, encoding="cp1251")
print(likvid_data.shape[0])
print(otozv_data.shape[0])

# Подгружаем табличку с данными.
bank_frame = pd.read_csv(REPORTING_DATA_FOLDER + 'reporting_all.csv', sep=',',header=0, encoding="cp1251")

1176
503
2047


//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [81]:
bank_frame[['лицензия']] = bank_frame[['лицензия']].astype(int)
bank_frame['дата'] = pd.to_datetime(bank_frame['дата'])

# Норматив ликвидности банка Н3 

In [82]:
liquidity = pd.read_csv(MANUALLY_PARSED_DATA_FOLDER + 'bank_liquidity.csv', sep=',', header = 0,  encoding="cp1251")

In [83]:
liquidity.dropna(axis=0, how='any', inplace = True)
liquidity.columns = ['drop', 'лицензия', 'N3_early', 'дата']
liquidity[['лицензия']] = liquidity[['лицензия']].astype(int)
liquidity['дата'] = pd.to_datetime(liquidity['дата'])

liquidity.drop(['drop'], inplace = True, axis = 1)

In [84]:
liquidity.head(5)

,лицензия,N3_early,дата
0,1,73.16,2010-06-01
1,67,63.10,2010-06-01
2,85,90.30,2010-06-01
3,96,143.33,2010-06-01
4,101,147.25,2010-06-01


In [85]:
liquidity['дата'].max()

Timestamp('2015-01-01 00:00:00')

# Обработка данных по закрытым банкам

In [86]:
# Данные скачались немного криво: надо поменять день и месяц местами
closed_banks = pd.read_csv(CLOSED_BANKS_DATA_FOLDER + 'closed_banks.csv',
                           sep=',', header = 0,  encoding="cp1251")

def change_month_day(date):
    
    y = date.year
    m = date.month
    if len(str(m)) == 1:
        m = '-0'+ str(m)
    else:
        m = '-' + str(m)
    d = '-01'
    date = str(y) + m + d
    date = pd.to_datetime(date)
    
    return date

closed_banks['ликвдата'] = pd.to_datetime(closed_banks['ликвдата'])
closed_banks['дата'] = closed_banks['ликвдата'].apply(change_month_day)

# Оставляем только столбца "лицензия" и "дата"
closed_banks = closed_banks[['Лицензия','дата']]
closed_banks.columns = ['лицензия','дата']
closed_banks['default'] = 1

In [87]:
# Очистка данных
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-К', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-Д', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-Р', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-K', '')
closed_banks['лицензия'] = closed_banks['лицензия'].str.replace('-к', '')
closed_banks = closed_banks.iloc[ :(closed_banks.shape[0] - 2), :]
closed_banks[['лицензия']] = closed_banks[['лицензия']].astype(int)
closed_banks['дата'] = pd.to_datetime(closed_banks['дата'])

In [88]:
closed_banks.head()

,лицензия,дата,default
0,2491,2018-02-01,1
1,2145,2018-02-01,1
2,3011,2018-02-01,1
3,2519,2018-02-01,1
4,3295,2018-12-01,1


# Макро показатели

In [89]:
macro = pd.read_csv(MANUALLY_PARSED_DATA_FOLDER + 'all_macroeconomic_factors.csv',
                    sep=';', header = 0,  encoding="cp1251")

macro.drop(['Unnamed: 0'], axis = 1, inplace = True)

# создадаи те же даты
years = [str(item) for item in range(2010,2018)]
months = [str(item) for item in range(1,13)]
months2 = [ ]
for item in months:
    if len(item)==1:
        item = '0'+item
    months2.append(item)
dats = [ ]
for item in years:
    for jtem in months2:
        dats.append(item + '-' + jtem + '-01')

dats.append('2018-01-01')
dats = pd.to_datetime(dats)
macro['Date'] = dats


columns_ok = ['дата', 'INF_SA', 'NX_growth', 'micex_std', 'miacr_std', 'miacr_amount',
        'usd_rub_std_diff', 'micex_return', 'net_foreign_assets_diff',
        'net_gov_debt_diff', 'other_fin_debt_diff',
        'retail_debt_SA_DETREND_diff', 'stocks_capital_diff',
        'i_retail_spread_diff', 'usd_rub_return', 'miacr_diff']
macro.columns = columns_ok

macro.head()

,дата,INF_SA,NX_growth,micex_std,miacr_std,miacr_amount,usd_rub_std_diff,micex_return,net_foreign_assets_diff,net_gov_debt_diff,other_fin_debt_diff,retail_debt_SA_DETREND_diff,stocks_capital_diff,i_retail_spread_diff,usd_rub_return,miacr_diff
0,2010-01-01,0.020000,0,32.00000,1.181512,2643995,1.081969,0.000000,-156425,1595305,47521,33587.30532,214757,0.0,0.008065,-0.830116
1,2010-02-01,0.020000,2728,29.75618,0.724981,2643995,1.081969,0.000000,-156425,1595305,47521,33587.30532,214757,0.0,0.008065,-0.830116
2,2010-03-01,0.027475,-980,40.07545,0.330918,2643995,-0.527030,-0.057148,225429,-265831,-34392,22186.02349,59909,-0.1,0.010699,-0.172562
3,2010-04-01,0.024986,-299,23.77384,0.359785,2643995,0.345271,0.043424,-128438,430871,-21876,-14983.52885,4293,0.1,-0.019848,-0.296650
4,2010-05-01,0.021233,-767,25.82706,0.246044,2643995,-0.334974,0.048447,-223667,92737,-1075,-19523.94108,64243,-0.1,-0.012387,-0.255000


# Капитал банков

In [90]:
capital = pd.read_csv(MANUALLY_PARSED_DATA_FOLDER + 'bank_capital.csv',
                      sep=',', header = 0,  encoding="cp1251")

capital.columns = ['drop', 'date', 'bank', 'license', 'capital', 'msk_spb']
capital.drop(['drop'], inplace = True, axis = 1)

capital.dropna(axis=0, how='any', inplace = True)
capital[['license']] = capital[['license']].astype(int)
capital['date'] = pd.to_datetime(capital['date'])

capital.columns = ['дата', 'банк', 'лицензия', 'capital', 'msk_spb']

capital.tail(5)

,дата,банк,лицензия,capital,msk_spb
75452,2017-12-01,Бинбанк,323,-36449784.0,1
75453,2017-12-01,Московский Областной Банк,1751,-125417931.0,1
75454,2017-12-01,Национальный Банк «Траст»,3279,-207524772.0,1
75455,2017-12-01,Банк «ФК Открытие»,2209,-209484321.0,1
75456,2017-12-01,Рост Банк,2888,-367608967.0,1


# Join всего в один датафрейм

In [91]:
bank_frame.sort_values(by = ['лицензия', 'дата'], inplace = True)
capital.sort_values(by = ['лицензия', 'дата'], inplace = True)
liquidity.sort_values(by = ['лицензия', 'дата'], inplace = True)
macro.sort_values(by = ['дата'], inplace = True)

In [92]:
df = bank_frame.set_index(['лицензия', 'дата']).\
        join(capital.set_index(['лицензия', 'дата']), how = 'inner').\
        join(macro.set_index(['дата']), how = 'inner').\
        join(liquidity.set_index(['лицензия', 'дата']), how = 'left').\
        join(closed_banks.set_index(['лицензия', 'дата']), how = 'left')

df.drop(['Лицензия_банк'], inplace = True, axis = 1)

### Clean data

In [93]:
# очистка данных:

text_cleansing = lambda x: x.replace('?', '-').replace(' ', '').replace(',','.') 

items = ['10', '1000','110','120','130', '140','160', '260', '30', 
        '300', '360','40', '400', '50', '500', '60', '70', '1100', 
        '200','1300', '1400', '1500', '401', '1800', '1700','1600']

items = ["ИД_" + i for i in items]

for col_name in items:
    df[col_name] = df[col_name].astype(str)
    df[col_name] = df[col_name].apply(text_cleansing)
    
df[items] = df[items].astype(float)

In [94]:
# часть данных оказалась в долях, а не в %: исправим это
df['ИД_1000'] = df['ИД_1000'].astype(float) + (df['ИД_1000'].astype(float) < 0) * df['ИД_1000'].astype(float)*99
df['ИД_1100'] = df['ИД_1100'].astype(float) + (df['ИД_1100'].astype(float) < 0) * df['ИД_1100'].astype(float)*99

In [95]:
df.reset_index(inplace=True)

In [96]:
# уберу ненужные столбцы
df.drop(['банк'], inplace = True, axis = 1)

### Fill NaNs

In [98]:
df['ИД_1800'] = df['ИД_1800'].fillna(0).astype(int)
df['N3_early'] = df['N3_early'].fillna(0).astype(int)
df['default'] = df['default'].fillna(0)   # if no info on bankrupcy, then no bankruptcy

In [99]:
df.dropna(thresh = 16, 
          subset = ['ИД_10', 'ИД_1000','ИД_110','ИД_120','ИД_130', 'ИД_140','ИД_160', 'ИД_260',\
                                  'ИД_30', 'ИД_300', 'ИД_360','ИД_40', 'ИД_400', 'ИД_50', 'ИД_500', 'ИД_60',\
                                  'ИД_70', 'ИД_1100'],
          inplace = True)

In [100]:
# Заполним пропуски медианой
from sklearn.preprocessing import Imputer

licence = df['лицензия']
date = df['дата']

df.drop(['лицензия', 'дата'], axis=1, inplace=True)

df_columns = df.columns

imr = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0)
imr.fit(df)
df = imr.transform(df.values)

//anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


## Финальные шаги

In [101]:
# Добавим индексы
df = pd.DataFrame(df, columns = df_columns)
df['лицензия'] = licence
df['дата'] = date
df = df.set_index(['лицензия', 'дата'])
df.head(5)

ИД_10  ИД_1000    ИД_110    ИД_120  ИД_130  ИД_140  \
лицензия дата                                                                
0.0      2010-02-01  423017.0     27.0  112770.0   60000.0     0.0     0.0   
         2010-03-01  498411.0     75.0  172628.0   90000.0     0.0     0.0   
         2010-04-01  571220.0     54.0  211860.0   90000.0     0.0     0.0   
         2010-05-01  523027.0     41.0  159970.0   90000.0     0.0     0.0   
         2010-06-01  473713.0     31.0  131782.0  135000.0     0.0     0.0   

                     ИД_160  ИД_260  ИД_30    ИД_300  ...  \
лицензия дата                                         ...   
0.0      2010-02-01     0.0     0.0  102.0  225000.0  ...   
         2010-03-01     0.0     0.0  569.0  225000.0  ...   
         2010-04-01     0.0   997.0  651.0  227287.0  ...   
         2010-05-01     0.0   481.0  677.0  227130.0  ...   
         2010-06-01     0.0   485.0  644.0  191306.0  ...   

                     net_foreign_assets_diff  net_gov_debt_diff  \
лицензия дата                                                     
0.0      2010-02-01                -156425.0          1595305.0   
         2010-03-01                 225429.0          -265831.0   
         2010-04-01                -128438.0           430871.0   
         2010-05-01                -223667.0            92737.0   
         2010-06-01                 251702.0          -134963.0   

                     other_fin_debt_diff  retail_debt_SA_DETREND_diff  \
лицензия дата                                                           
0.0      2010-02-01              47521.0                  33587.30532   
         2010-03-01             -34392.0                  22186.02349   
         2010-04-01             -21876.0                 -14983.52885   
         2010-05-01              -1075.0                 -19523.94108   
         2010-06-01                 81.0                 -20743.43746   

                     stocks_capital_diff  i_retail_spread_diff  \
лицензия дата                                                    
0.0      2010-02-01             214757.0                   0.0   
         2010-03-01              59909.0                  -0.1   
         2010-04-01               4293.0                   0.1   
         2010-05-01              64243.0                  -0.1   
         2010-06-01              36008.0                   0.1   

                     usd_rub_return  miacr_diff  N3_early  default  
лицензия дата                                                       
0.0      2010-02-01        0.008065   -0.830116       0.0      0.0  
         2010-03-01        0.010699   -0.172562       0.0      0.0  
         2010-04-01       -0.019848   -0.296650       0.0      0.0  
         2010-05-01       -0.012387   -0.255000       0.0      0.0  
         2010-06-01        0.042533   -0.533087       0.0      0.0  

[5 rows x 45 columns]

In [102]:
df.columns

Index(['ИД_10', 'ИД_1000', 'ИД_110', 'ИД_120', 'ИД_130', 'ИД_140', 'ИД_160',
       'ИД_260', 'ИД_30', 'ИД_300', 'ИД_360', 'ИД_40', 'ИД_400', 'ИД_50',
       'ИД_500', 'ИД_60', 'ИД_70', 'ИД_1100', 'ИД_200', 'ИД_1300', 'ИД_1400',
       'ИД_1500', 'ИД_401', 'ИД_1800', 'ИД_1700', 'ИД_1600', 'capital',
       'msk_spb', 'INF_SA', 'NX_growth', 'micex_std', 'miacr_std',
       'miacr_amount', 'usd_rub_std_diff', 'micex_return',
       'net_foreign_assets_diff', 'net_gov_debt_diff', 'other_fin_debt_diff',
       'retail_debt_SA_DETREND_diff', 'stocks_capital_diff',
       'i_retail_spread_diff', 'usd_rub_return', 'miacr_diff', 'N3_early',
       'default'],
      dtype='object')

In [103]:
columns = ['net_assets', 'ROA', 'liquid', 'ibl', 'stocks', 'bond', 'oth_cap', 'sunk_retail_credit',\
          'NI', 'organization_credit', 'sunk_organization_credit', 'credit_portf', 'drop', 'sunk_credit_portf',\
          'organization_deposit', 'retail_deposit', 'security_tot', 'ROE', 'retail_credit', 'reserv_credit_perc',\
          'zalog_credit_perc', 'foreign_na_fr', 'retail_deposit_fr', 'N3', 'N2', 'N1', 
           
           'capital',
           'msk_spb', 'INF_SA', 'NX_growth', 'micex_std', 'miacr_std',
           'miacr_amount', 'usd_rub_std_diff', 'micex_return',
           'net_foreign_assets_diff', 'net_gov_debt_diff', 'other_fin_debt_diff',
           'retail_debt_SA_DETREND_diff', 'stocks_capital_diff',
           'i_retail_spread_diff', 'usd_rub_return', 'miacr_diff', 'N3_early',
           'default']

In [104]:
df.columns = columns
df.drop('drop', axis = 1, inplace = True)

In [111]:
# N3 существует толбко ПОСЛЕ 2015 года, до 2015 этот столбец можно взять из N3_early

df.loc[(df['N3'] == 0) & (df['N3_early'] != 0), 'N3'] = df.loc[(df['N3'] == 0) & (df['N3_early'] != 0), 'N3_early'].values
df.drop('N3_early', axis=1, inplace=True)

In [113]:
df.head(5)

net_assets   ROA    liquid       ibl  stocks  bond  \
лицензия дата                                                             
0.0      2010-02-01    423017.0  27.0  112770.0   60000.0     0.0   0.0   
         2010-03-01    498411.0  75.0  172628.0   90000.0     0.0   0.0   
         2010-04-01    571220.0  54.0  211860.0   90000.0     0.0   0.0   
         2010-05-01    523027.0  41.0  159970.0   90000.0     0.0   0.0   
         2010-06-01    473713.0  31.0  131782.0  135000.0     0.0   0.0   

                     oth_cap  sunk_retail_credit     NI  organization_credit  \
лицензия дата                                                                  
0.0      2010-02-01      0.0                 0.0  102.0             225000.0   
         2010-03-01      0.0                 0.0  569.0             225000.0   
         2010-04-01      0.0               997.0  651.0             227287.0   
         2010-05-01      0.0               481.0  677.0             227130.0   
         2010-06-01      0.0               485.0  644.0             191306.0   

                     ...  micex_return  net_foreign_assets_diff  \
лицензия дата        ...                                          
0.0      2010-02-01  ...      0.000000                -156425.0   
         2010-03-01  ...     -0.057148                 225429.0   
         2010-04-01  ...      0.043424                -128438.0   
         2010-05-01  ...      0.048447                -223667.0   
         2010-06-01  ...     -0.105536                 251702.0   

                     net_gov_debt_diff  other_fin_debt_diff  \
лицензия дата                                                 
0.0      2010-02-01          1595305.0              47521.0   
         2010-03-01          -265831.0             -34392.0   
         2010-04-01           430871.0             -21876.0   
         2010-05-01            92737.0              -1075.0   
         2010-06-01          -134963.0                 81.0   

                     retail_debt_SA_DETREND_diff  stocks_capital_diff  \
лицензия дата                                                           
0.0      2010-02-01                  33587.30532             214757.0   
         2010-03-01                  22186.02349              59909.0   
         2010-04-01                 -14983.52885               4293.0   
         2010-05-01                 -19523.94108              64243.0   
         2010-06-01                 -20743.43746              36008.0   

                     i_retail_spread_diff  usd_rub_return  miacr_diff  default  
лицензия дата                                                                   
0.0      2010-02-01                   0.0        0.008065   -0.830116      0.0  
         2010-03-01                  -0.1        0.010699   -0.172562      0.0  
         2010-04-01                   0.1       -0.019848   -0.296650      0.0  
         2010-05-01                  -0.1       -0.012387   -0.255000      0.0  
         2010-06-01                   0.1        0.042533   -0.533087      0.0  

[5 rows x 43 columns]

In [115]:
df.to_csv('prepared_data/dataset.csv',
          sep=',',header=True, index=True)